# SQL скрипт для БД

In [5]:
command = '''-- Создание пользовательских типов ENUM
CREATE TYPE business_sector_enum AS ENUM (
    'Информационные технологии', 
    'Финансовые организации', 
    'Коммерция', 
    'Энергетика', 
    'Производство', 
    'Недвижимость'
);

CREATE TYPE product_name_enum AS ENUM ('Потребительский кредит', 'Ипотека', 'Автокредит', 'Кредитная карта', 'Бизнес-кредит', 'Возобновляемая кредитная линия', 'Гарантия'
);

CREATE TYPE credit_status_enum AS ENUM ('Активный', 'Закрытый', 'Дефолт');

CREATE TYPE transaction_type_enum AS ENUM ('Выплата основного долга', 'Выплата просроченного долга', 'Выплата штрафов'
);

CREATE TYPE default_reason_enum AS ENUM (
    'Просрочка', 'Банкротство', 'Реструктуризация', 'Ликвидация', 'Цессия'
);

CREATE TYPE reserve_stage_enum AS ENUM ('Стадия 1', 'Стадия 2', 'Стадия 3');

-- Создание таблиц
CREATE TABLE individual_borrowers (
    individual_id SERIAL PRIMARY KEY,
    date_of_birth DATE
);

CREATE TABLE company_borrowers (
    company_id SERIAL PRIMARY KEY,
    company_name VARCHAR(100) NOT NULL,
    business_sector business_sector_enum NOT NULL
);

CREATE TABLE credit_products (
    product_id SERIAL PRIMARY KEY,
    product_name product_name_enum NOT NULL,
    interest_rate DECIMAL(5, 2) NOT NULL,
    min_loan_amount DECIMAL(15, 2) NOT NULL,
    max_loan_amount DECIMAL(15, 2) NOT NULL,
    term_months INT NOT NULL
);

CREATE TABLE credits (
    credit_id SERIAL PRIMARY KEY,
    individual_id INT,
    company_id INT,
    product_id INT NOT NULL,
    loan_amount DECIMAL(15, 2) NOT NULL,
    interest_rate DECIMAL(5, 2) NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE,
    status credit_status_enum DEFAULT 'Активный',
    FOREIGN KEY (individual_id) REFERENCES individual_borrowers(individual_id),
    FOREIGN KEY (company_id) REFERENCES company_borrowers(company_id),
    FOREIGN KEY (product_id) REFERENCES credit_products(product_id)
);

CREATE TABLE client_transactions (
    transaction_id SERIAL PRIMARY KEY,
    credit_id INT NOT NULL,
    transaction_type transaction_type_enum NOT NULL,
    amount DECIMAL(15, 2) NOT NULL,
    transaction_date TIMESTAMP NOT NULL,
    FOREIGN KEY (credit_id) REFERENCES credits(credit_id)
);

CREATE TABLE risk_ratings (
    rating_id SERIAL PRIMARY KEY,
    individual_id INT,
    company_id INT,
    credit_id INT NOT NULL,
    rating INT CHECK (rating BETWEEN 1 AND 26) NOT NULL,
    probability_of_default DECIMAL(5, 4) NOT NULL,
    rating_date DATE NOT NULL,
    FOREIGN KEY (individual_id) REFERENCES individual_borrowers(individual_id),
    FOREIGN KEY (company_id) REFERENCES company_borrowers(company_id),
    FOREIGN KEY (credit_id) REFERENCES credits(credit_id)
);

CREATE TABLE defaults (
    default_id SERIAL PRIMARY KEY,
    credit_id INT NOT NULL,
    default_date DATE NOT NULL,
    default_amount DECIMAL(15, 2) NOT NULL,
    recovery_amount DECIMAL(15, 2) NOT NULL,
    reason_for_default default_reason_enum NOT NULL,
    FOREIGN KEY (credit_id) REFERENCES credits(credit_id)
);

CREATE TABLE credit_reserves (
    reserve_id SERIAL PRIMARY KEY,
    credit_id INT NOT NULL,
    reserve_stage reserve_stage_enum NOT NULL,
    reserve_amount DECIMAL(15, 2) NOT NULL,
    FOREIGN KEY (credit_id) REFERENCES credits(credit_id)
);'''

In [6]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "mydb",
    "user": "myuser",
    "password": "secure_password"
}


with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cursor:
        cursor.execute(command)
        conn.commit()

# Генерация данных

In [15]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker for generating fake data
fake = Faker()

# Set random seed for reproducibility
np.random.seed(42)

# Define the number of rows for each table
min_rows = 100
max_rows = 10000

# Helper function to generate random dates within a range
def random_date(start_date, end_date):
    return start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days))

# Generate IndividualBorrowers DataFrame
def generate_individual_borrowers():
    num_rows = np.random.randint(min_rows, max_rows + 1)
    data = {
        "IndividualID": range(1, num_rows + 1),
        "DateOfBirth": [random_date(datetime(1900, 1, 1), datetime.today()) for _ in range(num_rows)]
    }
    return pd.DataFrame(data)

# Generate CompanyBorrowers DataFrame
def generate_company_borrowers():
    num_rows = np.random.randint(min_rows, max_rows + 1)
    sectors = ['Информационные технологии', 'Финансовые организации', 'Коммерция', 'Энергетика', 'Производство', 'Недвижимость']
    data = {
        "CompanyID": range(1, num_rows + 1),
        "CompanyName": [fake.company() for _ in range(num_rows)],
        "BusinessSector": np.random.choice(sectors, num_rows)
    }
    return pd.DataFrame(data)

# Generate CreditProducts DataFrame
def generate_credit_products():
    num_rows = np.random.randint(min_rows, max_rows + 1)
    products = ['Потребительский кредит', 'Ипотека', 'Автокредит', 'Кредитная карта', 'Бизнес-кредит', 'Возобновляемая кредитная линия', 'Гарантия']
    data = {
        "ProductID": range(1, num_rows + 1),
        "ProductName": np.random.choice(products, num_rows),
        "InterestRate": np.round(np.random.uniform(0.01, 0.3, num_rows), 2),
        "MinLoanAmount": np.round(np.random.uniform(1000, 100000, num_rows), 2),
        "MaxLoanAmount": np.round(np.random.uniform(100000, 1000000, num_rows), 2),
        "TermMonths": np.random.randint(12, 360, num_rows)
    }
    return pd.DataFrame(data)



# Generate Credits DataFrame
def generate_credits(individual_borrowers, company_borrowers, credit_products):
    num_rows = np.random.randint(min_rows, max_rows + 1)
    data = {
        "CreditID": range(1, num_rows + 1),
        "IndividualID": [None] * num_rows,  # Initialize as None
        "CompanyID": [None] * num_rows,    # Initialize as None
        "ProductID": [None] * num_rows,    # Initialize as None
        "LoanAmount": np.round(np.random.uniform(1000, 1000000, num_rows), 2),
        "InterestRate": np.round(np.random.uniform(0.01, 0.3, num_rows), 2),
        "StartDate": [random_date(datetime(2010, 1, 1), datetime.today()) for _ in range(num_rows)],
        "EndDate": [random_date(datetime(2010, 1, 1), datetime.today() + timedelta(days=365 * 10)) for _ in range(num_rows)],
        "Status": np.random.choice(['Активный', 'Закрытый', 'Дефолт'], num_rows)
    }

    # Define product categories for individual and company borrowers
    individual_products = ['Потребительский кредит', 'Ипотека', 'Автокредит', 'Кредитная карта']
    company_products = ['Бизнес-кредит', 'Возобновляемая кредитная линия', 'Гарантия']

    # Filter ProductIDs for individual and company products
    individual_product_ids = credit_products[credit_products['ProductName'].isin(individual_products)]['ProductID'].tolist()
    company_product_ids = credit_products[credit_products['ProductName'].isin(company_products)]['ProductID'].tolist()

    for i in range(num_rows):
        if np.random.rand() > 0.5:
            # Assign to individual borrower
            data["IndividualID"][i] = np.random.choice(individual_borrowers["IndividualID"])
            data["ProductID"][i] = np.random.choice(individual_product_ids)
        else:
            # Assign to company borrower
            data["CompanyID"][i] = np.random.choice(company_borrowers["CompanyID"])
            data["ProductID"][i] = np.random.choice(company_product_ids)

    return pd.DataFrame(data)


# Generate ClientTransactions DataFrame
def generate_client_transactions(credits):
    num_rows = np.random.randint(min_rows, max_rows + 1)
    transaction_types = ['Выплата основного долга', 'Выплата просроченного долга', 'Выплата штрафов']
    data = {
        "TransactionID": range(1, num_rows + 1),
        "CreditID": np.random.choice(credits["CreditID"], num_rows),
        "TransactionType": np.random.choice(transaction_types, num_rows),
        "Amount": np.round(np.random.uniform(100, 100000, num_rows), 2),
        "TransactionDate": [random_date(datetime(2010, 1, 1), datetime.today()) for _ in range(num_rows)]
    }
    return pd.DataFrame(data)

# Generate RiskRatings DataFrame
def generate_risk_ratings(individual_borrowers, company_borrowers, credits):
    num_rows = np.random.randint(min_rows, max_rows + 1)
    data = {
        "RatingID": range(1, num_rows + 1),
        "IndividualID": list(np.random.choice(individual_borrowers["IndividualID"], num_rows)),
        "CompanyID": list(np.random.choice(company_borrowers["CompanyID"], num_rows)),
        "CreditID": np.random.choice(credits["CreditID"], num_rows),
        "Rating": np.random.randint(1, 27, num_rows),
        "ProbabilityOfDefault": np.round(np.random.uniform(0.0, 1.0, num_rows), 4),
        "RatingDate": [random_date(datetime(2010, 1, 1), datetime.today()) for _ in range(num_rows)]
    }
    # Ensure either IndividualID or CompanyID is populated, but not both
    for i in range(num_rows):
        if np.random.rand() > 0.5:
            data["CompanyID"][i] = None
        else:
            data["IndividualID"][i] = None
    return pd.DataFrame(data)

# Generate Defaults DataFrame
def generate_defaults(credits):
    num_rows = np.random.randint(min_rows, max_rows + 1)
    reasons = ['Просрочка', 'Банкротство', 'Реструктуризация', 'Ликвидация', 'Цессия']
    data = {
        "DefaultID": range(1, num_rows + 1),
        "CreditID": np.random.choice(credits["CreditID"], num_rows),
        "DefaultDate": [random_date(datetime(2010, 1, 1), datetime.today()) for _ in range(num_rows)],
        "DefaultAmount": np.round(np.random.uniform(1000, 1000000, num_rows), 2),
        "RecoveryAmount": np.round(np.random.uniform(0, 1000000, num_rows), 2),
        "ReasonForDefault": np.random.choice(reasons, num_rows)
    }
    return pd.DataFrame(data)

# Generate CreditReserves DataFrame
def generate_credit_reserves(credits):
    num_rows = np.random.randint(min_rows, max_rows + 1)
    stages = ['Стадия 1', 'Стадия 2', 'Стадия 3']
    data = {
        "ReserveID": range(1, num_rows + 1),
        "CreditID": np.random.choice(credits["CreditID"], num_rows),
        "ReserveStage": np.random.choice(stages, num_rows),
        "ReserveAmount": np.round(np.random.uniform(1000, 1000000, num_rows), 2)
    }
    return pd.DataFrame(data)

# Generate all DataFrames
individual_borrowers = generate_individual_borrowers()
company_borrowers = generate_company_borrowers()
credit_products = generate_credit_products()
credits = generate_credits(individual_borrowers, company_borrowers, credit_products)
client_transactions = generate_client_transactions(credits)
risk_ratings = generate_risk_ratings(individual_borrowers, company_borrowers, credits)
defaults = generate_defaults(credits)
credit_reserves = generate_credit_reserves(credits)

# Display the first few rows of each DataFrame
print("IndividualBorrowers:")
print(individual_borrowers.head())
print("\nCompanyBorrowers:")
print(company_borrowers.head())
print("\nCreditProducts:")
print(credit_products.head())
print("\nCredits:")
print(credits.head())
print("\nClientTransactions:")
print(client_transactions.head())
print("\nRiskRatings:")
print(risk_ratings.head())
print("\nDefaults:")
print(defaults.head())
print("\nCreditReserves:")
print(credit_reserves.head())

IndividualBorrowers:
   IndividualID DateOfBirth
0             1  1943-04-01
1             2  1902-05-11
2             3  2004-06-22
3             4  2022-06-22
4             5  1930-11-24

CompanyBorrowers:
   CompanyID                   CompanyName             BusinessSector
0          1         Jones, Smith and Hall                  Коммерция
1          2                     Hayes Ltd  Информационные технологии
2          3   Nelson, Francis and Gardner                 Энергетика
3          4  Thomas, Potter and Velasquez     Финансовые организации
4          5  Steele, Mitchell and Vasquez  Информационные технологии

CreditProducts:
   ProductID             ProductName  InterestRate  MinLoanAmount  \
0          1  Потребительский кредит          0.20       58472.13   
1          2                Гарантия          0.16       46405.43   
2          3                Гарантия          0.12       55424.49   
3          4  Потребительский кредит          0.30       38234.80   
4         

# Вставка данных

In [18]:
import psycopg2
from psycopg2.extras import execute_batch
from psycopg2 import sql
import pandas as pd
import numpy as np

def insert_individual_borrowers(df, conn):
    with conn.cursor() as cursor:
        df = df.where(pd.notnull(df), None)
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO individual_borrowers (individual_id, date_of_birth)
            VALUES (%s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(individual_id) FROM individual_borrowers;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE individual_borrowers_individual_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1))
            )
        conn.commit()

def insert_company_borrowers(df, conn):
    with conn.cursor() as cursor:
        df = df.where(pd.notnull(df), None)
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO company_borrowers (company_id, company_name, business_sector)
            VALUES (%s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(company_id) FROM company_borrowers;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE company_borrowers_company_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

def insert_credit_products(df, conn):
    with conn.cursor() as cursor:
        df = df.where(pd.notnull(df), None)
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO credit_products (product_id, product_name, interest_rate, 
                                      min_loan_amount, max_loan_amount, term_months)
            VALUES (%s, %s, %s, %s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(product_id) FROM credit_products;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE credit_products_product_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

def insert_credits(df, conn):
    with conn.cursor() as cursor:
        df = df.replace({np.nan: None})
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO credits (credit_id, individual_id, company_id, product_id, 
                                loan_amount, interest_rate, start_date, end_date, status)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(credit_id) FROM credits;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE credits_credit_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

def insert_client_transactions(df, conn):
    with conn.cursor() as cursor:
        df = df.where(pd.notnull(df), None)
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO client_transactions (transaction_id, credit_id, transaction_type, 
                                          amount, transaction_date)
            VALUES (%s, %s, %s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(transaction_id) FROM client_transactions;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE client_transactions_transaction_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

def insert_risk_ratings(df, conn):
    with conn.cursor() as cursor:
        df = df.replace({np.nan: None})
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO risk_ratings (rating_id, individual_id, company_id, 
                                   credit_id, rating, probability_of_default, rating_date)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(rating_id) FROM risk_ratings;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE risk_ratings_rating_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

def insert_defaults(df, conn):
    with conn.cursor() as cursor:
        df = df.where(pd.notnull(df), None)
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO defaults (default_id, credit_id, default_date, 
                                default_amount, recovery_amount, reason_for_default)
            VALUES (%s, %s, %s, %s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(default_id) FROM defaults;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE defaults_default_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

def insert_credit_reserves(df, conn):
    with conn.cursor() as cursor:
        df = df.where(pd.notnull(df), None)
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        
        query = sql.SQL("""
            INSERT INTO credit_reserves (reserve_id, credit_id, reserve_stage, reserve_amount)
            VALUES (%s, %s, %s, %s)
        """)
        
        execute_batch(cursor, query, data)
        
        cursor.execute("SELECT MAX(reserve_id) FROM credit_reserves;")
        max_id = cursor.fetchone()[0]
        if max_id:
            cursor.execute(
                sql.SQL("ALTER SEQUENCE credit_reserves_reserve_id_seq RESTART WITH {};")
                .format(sql.Literal(max_id + 1)))
        conn.commit()

In [19]:
conn = psycopg2.connect(
    dbname="mydb",
    user="myuser",
    password="secure_password",
    host="localhost"
)

insert_individual_borrowers(individual_borrowers, conn)
insert_company_borrowers(company_borrowers, conn)
insert_credit_products(credit_products, conn)
insert_credits(credits, conn)
insert_client_transactions(client_transactions, conn)
insert_risk_ratings(risk_ratings, conn)
insert_defaults(defaults, conn)
insert_credit_reserves(credit_reserves, conn)